In [1]:
import numpy as np
import webbrowser
from aocd.models import Puzzle
import re
import itertools
from scipy.ndimage.filters import minimum_filter
from scipy.ndimage import label
import matplotlib.pyplot as plt
from collections import defaultdict
import networkx as nx
%matplotlib inline
puzzle = Puzzle(year=2021, day=16)

In [2]:
webbrowser.open(puzzle.url);

## Part 1

In [3]:
tests = [
    ("D2FE28", 6),
    ("38006F45291200", 9),
    ("EE00D40C823060", 14),
    ("8A004A801A8002F478", 16),
    ("620080001611562C8802118E34", 12),
    ("C0015000016115A2E0802F182340", 23),
    ("A0016C880162017C3686B18A3D4780", 31),
]

In [4]:
def hex_to_binary(hex: str) -> str:
    binary = ""
    for ch in hex:
        ch_binary = bin(int(ch, 16))[2:]
        if len(ch_binary) < 4:
            ch_binary = "0" * (4 - len(ch_binary)) + ch_binary
        binary += ch_binary
    return binary

In [5]:
class Parser:
    def __init__(self, packet_hex: str):
        self.packet_hex = packet_hex
        self.packet = hex_to_binary(packet_hex)
        self.version_numbers = []
    
    def next_binary(self, n_bits: int) -> str:
        binary = self.packet[:n_bits]
        self.packet = self.packet[n_bits:]
        return binary
    
    def next_int(self, n_bits: int) -> int:
        return int(self.next_binary(n_bits), 2)
    
    
    def parse_packet(self):
        version, type_id = self.parse_header()
        if type_id == 4:  # literal
            result = self.parse_literal()
        else:  # operator
            subpackets = self.parse_operator_content()
            if type_id == 0:
                result = np.sum(subpackets)
            elif type_id == 1:
                result = np.prod(subpackets)
            elif type_id == 2:
                result = np.min(subpackets)
            elif type_id == 3:
                result = np.max(subpackets)
            elif type_id == 5:
                result = int(subpackets[0] > subpackets[1])
            elif type_id == 6:
                result = int(subpackets[0] < subpackets[1])
            elif type_id == 7:
                result = int(subpackets[0] == subpackets[1])
        
        return result
        
    def parse_header(self):
        version = self.next_int(3)
        self.version_numbers.append(version)
        type_id = self.next_int(3)
        return version, type_id
    
    def parse_literal(self) -> int:
        literal = ""
        while len(self.packet) >= 5:
            group = self.next_binary(5)
            literal += group[1:]
            if group[0] == "0":
                break
        return int(literal, 2)
    
    def parse_operator_content(self) -> int:
        length_type_id = self.next_int(1)
        if length_type_id == 0:
            args = dict(n_bits = self.next_int(15))
        else:
            args = dict(n_packets = self.next_int(11))
        return self.parse_subpackets(**args)
    
    def parse_subpackets(self, n_bits = None, n_packets = None):
        subpackets = []
        if n_bits:
            remaining_length = len(self.packet) - n_bits
            while len(self.packet) > remaining_length:
                subpackets.append(self.parse_packet())
        else:
            while len(subpackets) < n_packets:
                subpackets.append(self.parse_packet())
        return subpackets

In [6]:
def part_1(input_data):
    p = Parser(input_data)
    p.parse_packet()
    return sum(p.version_numbers)

In [7]:
for test_input, test_output in tests:
    assert part_1(test_input) == test_output

In [8]:
result = part_1(puzzle.input_data)
result

843

In [211]:
puzzle.answer_a = result

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


## Part 2

In [9]:
tests2 = [
    ("C200B40A82", 3),
    ("04005AC33890", 54),
    ("880086C3E88112", 7),
    ("CE00C43D881120", 9),
    ("D8005AC2A8F0", 1),
    ("F600BC2D8F", 0),
    ("9C005AC2F8F0", 0),
    ("9C0141080250320F1802104A08", 1)
]

In [10]:
def part_2(input_data):
    return Parser(input_data).parse_packet()

In [11]:
for test_input, test_output in tests2:
    assert part_2(test_input) == test_output

In [12]:
result = part_2(puzzle.input_data)
result

5390807940351

In [221]:
puzzle.answer_b = result

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 16! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
